In [1]:
from PythonPSI.api import PSI
import pandas as pd
import time
from ipynb.fs.full.functions import *
import glob

### Notes
This function is pretty slow, because it takes a while to get all the needed data with the API, especially when calling a lot of urls. With 50 urls, it takes about `20 minutes` to run.

Not every url returns useable performance data, since the API has some restrictions. Some webpages require too many queries or the request simply fails. With 50 urls, about `30 urls` return useable data. The results can vary.

This function will need some refactoring at some point to enhance performance.

In [2]:
# returns a dict with the cls, fcp, fid and lcp from given websites dataframe with performance data
def GetSpecificPerformanceData(df):
    url = df.loc["CUMULATIVE_LAYOUT_SHIFT_SCORE","id"]
    cls = pd.DataFrame(df.loc["CUMULATIVE_LAYOUT_SHIFT_SCORE","metrics"]).percentile.iloc[0]
    fcp = pd.DataFrame(df.loc["FIRST_CONTENTFUL_PAINT_MS","metrics"]).percentile.iloc[0]
    fid = pd.DataFrame(df.loc["FIRST_INPUT_DELAY_MS","metrics"]).percentile.iloc[0]
    lcp = pd.DataFrame(df.loc["LARGEST_CONTENTFUL_PAINT_MS","metrics"]).percentile.iloc[0]
    results = {"URL":url,"FCP":fcp,"LCP":lcp,"FID":fid,"CLS":cls}
    return results

def FileExists(filename):
    try:
        with open(f"websitespeed/{filename}.csv"):
            return True
    except IOError:
        potential_csv_file_exists = 0
        return False

# returns dataframe with website performance data
def GetWebsiteSpeed(keyword,df):
    rows_list = []
    urls = df["Ur"]
    failed_urls = []

    for key,value in urls.iteritems():
        filename = "{}-{}".format(keyword,value.replace('.', '').replace('/', '').replace(':', ''))
        if FileExists(filename) is True:
            continue
        
        try:
            data = PSI(value, category='performance', locale='en', stratergy='desktop', metrics='loadingExperience')
        except:
            continue
        data_keys = list(data.keys())
        
        # don't add row to list when API call gives error or data is incomplete
        if data_keys != ['error']:
            if data_keys != ['initial_url']:
                performance = GetSpecificPerformanceData(pd.DataFrame(data))
                rows_list.append(performance)
                temp_row = []
                temp_row.append(performance)
                df = pd.DataFrame(temp_row)
                df.to_csv(r"websitespeed/{}-{}.csv".format(keyword,filename))          
        else:
            failed_urls.append({"URL":value})
            print('one failed url added')
    
    failed_df = pd.DataFrame(failed_urls)
    for key,value in failed_df.iteritems():
        filename = "{}-{}".format(keyword,value.replace('.', '').replace('/', '').replace(':', ''))
        if FileExists(filename) is True:
            continue
        # get data from Google Pagespeed Insights API
        try:
            data = PSI(value, category='performance', locale='en', stratergy='desktop', metrics='loadingExperience')
        except:
            continue
        
        data_keys = list(data.keys())
        
        # don't add row to list when API call gives error or data is incomplete
        if data_keys != ['error']:
            if data_keys != ['initial_url']:
                performance = GetSpecificPerformanceData(pd.DataFrame(data))
                rows_list.append(performance)
                temp_row = []
                temp_row.append(performance)
                df = pd.DataFrame(temp_row)
                df.to_csv(r"websitespeed/{}-{}.csv".format(keyword,filename))           
        else:
            rows_list.append({"URL":value,"FCP":"NaN","LCP":"NaN","FID":"NaN","CLS":"NaN"})            
    
    files = glob.glob("websitespeed/{}-*.csv".format(keyword))
    dfs = [pd.read_csv(f) for f in files]
    complete_df = pd.concat(dfs)
    return complete_df[["URL","FCP","LCP","FID","CLS"]]

In [9]:
# test run
# d = {'Ur': ["https://www.bouwmaat.com/bouwmaterialen", 
#             "https://www.bouwbestel.nl/bouwmaterialen.html",
#             "https://www.hornbach.nl/shop/Bouwmateriaal/S4471/artikeloverzicht.html",
#             "https://www.online-bouwmaterialen.nl/",
#             "https://www.bouwonline.com/",
#             "https://www.hetjagershuis.com/c-4257901/bouwmaterialen/",
#             "https://www.3mnederland.nl/3M/nl_NL/p/c/bouwmaterialen/",
#             "bouwmaterialenzeeland.nl,https://www.bouwmaterialenzeeland.nl/",
#             "https://www.boer-staphorst.nl/klussen-bouwen/bouwmaterialen",
#             "https://www.eco-bouwmaterialen.nl/",
#             "https://www.flexbouwmateriaal.nl/",
#             "https://www.breukers.nl/bouwmaterialen",
#             "https://www.bouwhof.nl/bouw/bouwmaterialen/",
#             "https://www.kombibouwmaterialen.nl/producten/bouwmaterialen/",
#             "https://www.ggoedkoop.nl/",
#             "https://www.groenebouwmaterialen.nl/"
#             "https://www.tweakers.net"
#            ]}

# df = pd.DataFrame(data=d)
# keyword = 'bouwmaterialen'
# joe = GetWebsiteSpeed(keyword, df)
# joe

one failed url added
one failed url added


,URL,FCP,LCP,FID,CLS
0,https://berkela.home.xs4all.nl,490,650,2,0
0,https://eppinga.nl/,2093,3327,4,9
0,https://noordbouwmaterialen.nl/,3558,4220,3,7
0,https://nos.nl/artikel/2380864-het-is-een-stuk...,528,1056,2,15
0,https://vandepol.info,1055,1766,4,2
...,...,...,...,...,...
0,https://www.verdouw.nu/,730,1858,2,1
0,https://www.vidaxl.nl,1856,3134,6,36
0,https://www.wienerberger.nl,885,1587,2,11
0,https://www.witzand.nl/,1108,2148,4,2


In [10]:
# joe.head(80)

,URL,FCP,LCP,FID,CLS
0,https://berkela.home.xs4all.nl,490,650,2,0
0,https://eppinga.nl/,2093,3327,4,9
0,https://noordbouwmaterialen.nl/,3558,4220,3,7
0,https://nos.nl/artikel/2380864-het-is-een-stuk...,528,1056,2,15
0,https://vandepol.info,1055,1766,4,2
...,...,...,...,...,...
0,https://www.verdouw.nu/,730,1858,2,1
0,https://www.vidaxl.nl,1856,3134,6,36
0,https://www.wienerberger.nl,885,1587,2,11
0,https://www.witzand.nl/,1108,2148,4,2
